In [0]:
%load_ext autoreload
%matplotlib inline
%autoreload 2



from IPython import display


from torch.utils.data import DataLoader
import torchvision
import torch
from torchvision import transforms, datasets

import tensorflow as tf
import logging
from tensorflow import layers

import numpy as np

import os
import zipfile



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
#!unzip blues.zip 
# !unzip 'content/blues.zip'

# !mkdir hello

!pwd



/content


In [0]:
#!ls
# !mkdir 'Data_blues'
# %cd 'Data_blues'
# !unzip blues.zip
# !mv '/content/blues.zip' '/content/main_data/images'


Archive:  blues.zip
   creating: blues/
  inflating: blues/blues6.png        
  inflating: blues/blues22.png       
  inflating: blues/blues36.png       
  inflating: blues/blues37.png       
  inflating: blues/blues23.png       
  inflating: blues/blues7.png        
  inflating: blues/blues5.png        
  inflating: blues/blues35.png       
  inflating: blues/blues21.png       
  inflating: blues/blues20.png       
  inflating: blues/blues34.png       
  inflating: blues/blues4.png        
  inflating: blues/blues0.png        
   creating: __MACOSX/
   creating: __MACOSX/blues/
  inflating: __MACOSX/blues/._blues0.png  
  inflating: blues/blues30.png       
  inflating: blues/.DS_Store         
  inflating: __MACOSX/blues/._.DS_Store  
  inflating: blues/blues24.png       
  inflating: blues/blues18.png       
  inflating: blues/blues19.png       
  inflating: blues/blues25.png       
  inflating: blues/blues31.png       
  inflating: blues/blues1.png        
  inflating: __MACOSX/blu

In [0]:
# !rm blues.zip
# %cd blues
# !mkdir images
# %cd images
# !mv '/images' './content/Data_blues/blues'
# !unzip blues.zip
# %cd images
# %cd images
# !mv 'content/blues.zip'  'content/Data_blues/blues/images'




Archive:  blues.zip
   creating: blues/
  inflating: blues/.DS_Store         
   creating: __MACOSX/
   creating: __MACOSX/blues/
  inflating: __MACOSX/blues/._.DS_Store  
   creating: blues/data/
  inflating: blues/data/blues6.png   
  inflating: blues/data/blues22.png  
  inflating: blues/data/blues36.png  
  inflating: blues/data/blues37.png  
  inflating: blues/data/blues23.png  
  inflating: blues/data/blues7.png   
  inflating: blues/data/blues5.png   
  inflating: blues/data/blues35.png  
  inflating: blues/data/blues21.png  
  inflating: blues/data/blues20.png  
  inflating: blues/data/blues34.png  
  inflating: blues/data/blues4.png   
  inflating: blues/data/blues0.png   
   creating: __MACOSX/blues/data/
  inflating: __MACOSX/blues/data/._blues0.png  
  inflating: blues/data/blues30.png  
  inflating: blues/data/blues24.png  
  inflating: blues/data/blues18.png  
  inflating: blues/data/blues19.png  
  inflating: blues/data/blues25.png  
  inflating: blues/data/blues31.png  

In [0]:
Image_Pixels = 558 * 544
batch_size = 100
# Noise_Size = 100
# DATA_FOLDER = '/blues'
# hello = '{}'.format(DATA_FOLDER)
# print (hello)




In [0]:
def load_dataset():
    data_path = 'blues'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return train_loader


In [0]:
dataset = load_dataset()
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_batches = len(dataset)

In [0]:
IMAGES_SHAPE = (64, 64, 3)
NOISE_SIZE = 100

In [0]:

def default_conv2d(inputs, filters):
    return layers.conv2d(
        inputs,
        filters=filters,
        kernel_size=4,
        strides=(2, 2),
        padding='same',
        data_format='channels_last',
        use_bias=False,
    )

def default_conv2d_transpose(inputs, filters):
    return layers.conv2d_transpose(
        inputs,
        filters=filters,
        kernel_size=4,
        strides=(2, 2),
        padding='same',
        data_format='channels_last',
        use_bias=False,
    )

def noise(n_rows, n_cols):
    return np.random.normal(size=(n_rows, n_cols))

In [0]:
def discriminator(x):
    with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
        with tf.variable_scope("conv1"):
            conv1 = default_conv2d(x, 128)
            conv1 = nn.leaky_relu(conv1,alpha=0.2)
        
        with tf.variable_scope("conv2"):
            conv2 = default_conv2d(conv1, 256)
            conv2 = layers.batch_normalization(conv2)
            conv2 = nn.leaky_relu(conv2,alpha=0.2)
            
        with tf.variable_scope("conv3"):
            conv3 = default_conv2d(conv2, 512)
            conv3 = layers.batch_normalization(conv3)
            conv3 = nn.leaky_relu(conv3,alpha=0.2)
            
        with tf.variable_scope("conv4"):
            conv4 = default_conv2d(conv3, 1024)
            conv4 = layers.batch_normalization(conv3)
            conv4 = nn.leaky_relu(conv3,alpha=0.2)
        
        with tf.variable_scope("linear"):
            linear = clayers.flatten(conv4)
            linear = clayers.fully_connected(linear, 1)
        
        with tf.variable_scope("out"):
            out = nn.sigmoid(linear)
    return out

In [0]:
def generator(z):
    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
        
        with tf.variable_scope("linear"):
            linear = layers.fully_connected(z, 1024 * 4 * 4)
            
        with tf.variable_scope("conv1_transp"):
            # Reshape as 4x4 images
            conv1 = tf.reshape(linear, (-1, 4, 4, 1024))
            conv1 = default_conv2d_transpose(conv1, 512)
            conv1 = layers.batch_normalization(conv1)
            conv1 = nn.relu(conv1)
        
        with tf.variable_scope("conv2_transp"):
            conv2 = default_conv2d_transpose(conv1, 256)
            conv2 = layers.batch_normalization(conv2)
            conv2 = nn.relu(conv2)
            
        with tf.variable_scope("conv3_transp"):
            conv3 = default_conv2d_transpose(conv2, 128)
            conv3 = layers.batch_normalization(conv3)
            conv3 = nn.relu(conv3)
            
        with tf.variable_scope("conv4_transp"):
            conv4 = default_conv2d_transpose(conv3, 3)
        
        with tf.variable_scope("out"):
            out = tf.tanh(conv4)
    return out

In [0]:
## Real Input
X = tf.placeholder(tf.float32, shape=(None, )+IMAGES_SHAPE)
## Latent Variables / Noise
Z = tf.placeholder(tf.float32, shape=(None, NOISE_SIZE))

In [0]:
# Generator
G_sample = generator(Z)
# Discriminator
D_real = discriminator(X)
D_fake = discriminator(G_sample)

AttributeError: ignored